In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
spark = (SparkSession.builder.appName("date_handling")
         .master("local[*]")
         .config("spark.executor.memory", "1g")
         .getOrCreate()
         )

In [15]:
# Create a CSV file
csv_data = """id,date_iso,date_dmy,date_mdy,timestamp
1,2023-01-15,15/01/2023,01/15/2023,2023-01-15 10:30:00
2,2023-05-20,20/05/2023,05/20/2023,2023-05-20 15:45:00
3,InvalidDate,31/02/2023,02/31/2023,InvalidTimestamp
4,,,
"""

# Save the CSV file
with open("dates_data.csv", "w") as f:
    f.write(csv_data)

In [16]:
# Sample data with multiple date formats
data = [
    (1, "2023-01-15", "15/01/2023", "01/15/2023", "2023-01-15 10:30:00"),
    (2, "2023-05-20", "20/05/2023", "05/20/2023", "2023-05-20 15:45:00"),
    (3, "InvalidDate", "31/02/2023", "02/31/2023", "InvalidTimestamp"),  # Invalid dates
    (4, None, None, None, None)  # Null values
]

# Define column names
columns = ["id", "date_iso", "date_dmy", "date_mdy", "timestamp"]

# Create DataFrame
df = spark.createDataFrame(data, schema=columns)

# Show the DataFrame
df.show(truncate=False)

+---+-----------+----------+----------+-------------------+
|id |date_iso   |date_dmy  |date_mdy  |timestamp          |
+---+-----------+----------+----------+-------------------+
|1  |2023-01-15 |15/01/2023|01/15/2023|2023-01-15 10:30:00|
|2  |2023-05-20 |20/05/2023|05/20/2023|2023-05-20 15:45:00|
|3  |InvalidDate|31/02/2023|02/31/2023|InvalidTimestamp   |
|4  |NULL       |NULL      |NULL      |NULL               |
+---+-----------+----------+----------+-------------------+



In [17]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- date_iso: string (nullable = true)
 |-- date_dmy: string (nullable = true)
 |-- date_mdy: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [18]:
# from pyspark.sql.types import *
# # StructType for the schema
# struct_schema = StructType([
#     StructField("id", IntegerType(), nullable=True),
#     StructField("date_iso", StringType(), nullable=True),
#     StructField("date_dmy", StringType(), nullable=True),
#     StructField("date_mdy", StringType(), nullable=True),
#     StructField("timestamp", StringType(), nullable=True)
# ])

In [19]:
# DDL String for the schema
ddl_schema = """
    id INT,
    date_iso DATE,
    date_dmy DATE,
    date_mdy DATE,
    timestamp TIMESTAMP
"""

# Read the CSV file into a DataFrame
df_file = spark.read.option("header", True).schema(ddl_schema).csv("dates_data.csv")

# Show the DataFrame
df_file.show(truncate=False)

+---+----------+--------+--------+-------------------+
|id |date_iso  |date_dmy|date_mdy|timestamp          |
+---+----------+--------+--------+-------------------+
|1  |2023-01-15|NULL    |NULL    |2023-01-15 10:30:00|
|2  |2023-05-20|NULL    |NULL    |2023-05-20 15:45:00|
|3  |NULL      |NULL    |NULL    |NULL               |
|4  |NULL      |NULL    |NULL    |NULL               |
+---+----------+--------+--------+-------------------+



In [20]:
df_file.printSchema()

root
 |-- id: integer (nullable = true)
 |-- date_iso: date (nullable = true)
 |-- date_dmy: date (nullable = true)
 |-- date_mdy: date (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [21]:
from pyspark.sql.functions import to_date

df = (df
      .withColumn('parsed_date_iso', to_date('date_iso', 'yyyy-MM-dd'))
      .withColumn('parsed_date_dmy', to_date('date_dmy', 'dd/MM/yyyy'))
      .withColumn('parsed_date_mdy', to_date('date_mdy', 'MM/dd/yyyy')))

In [22]:
df.show(truncate=False)

+---+-----------+----------+----------+-------------------+---------------+---------------+---------------+
|id |date_iso   |date_dmy  |date_mdy  |timestamp          |parsed_date_iso|parsed_date_dmy|parsed_date_mdy|
+---+-----------+----------+----------+-------------------+---------------+---------------+---------------+
|1  |2023-01-15 |15/01/2023|01/15/2023|2023-01-15 10:30:00|2023-01-15     |2023-01-15     |2023-01-15     |
|2  |2023-05-20 |20/05/2023|05/20/2023|2023-05-20 15:45:00|2023-05-20     |2023-05-20     |2023-05-20     |
|3  |InvalidDate|31/02/2023|02/31/2023|InvalidTimestamp   |NULL           |NULL           |NULL           |
|4  |NULL       |NULL      |NULL      |NULL               |NULL           |NULL           |NULL           |
+---+-----------+----------+----------+-------------------+---------------+---------------+---------------+



# Timestamp

In [24]:
from pyspark.sql.functions import *

In [26]:
df = df.withColumn('parsed_timestamp', to_timestamp(df.timestamp, 'yyyy-MM-dd HH:mm:ss'))
df.show()
df.printSchema()

+---+-----------+----------+----------+-------------------+---------------+---------------+---------------+-------------------+
| id|   date_iso|  date_dmy|  date_mdy|          timestamp|parsed_date_iso|parsed_date_dmy|parsed_date_mdy|   parsed_timestamp|
+---+-----------+----------+----------+-------------------+---------------+---------------+---------------+-------------------+
|  1| 2023-01-15|15/01/2023|01/15/2023|2023-01-15 10:30:00|     2023-01-15|     2023-01-15|     2023-01-15|2023-01-15 10:30:00|
|  2| 2023-05-20|20/05/2023|05/20/2023|2023-05-20 15:45:00|     2023-05-20|     2023-05-20|     2023-05-20|2023-05-20 15:45:00|
|  3|InvalidDate|31/02/2023|02/31/2023|   InvalidTimestamp|           NULL|           NULL|           NULL|               NULL|
|  4|       NULL|      NULL|      NULL|               NULL|           NULL|           NULL|           NULL|               NULL|
+---+-----------+----------+----------+-------------------+---------------+---------------+-------------

In [27]:
df = (df
      .withColumn('year', year('parsed_timestamp'))
      .withColumn('month', month('parsed_timestamp'))
      .withColumn('day', day('parsed_timestamp'))
      .withColumn('hour', hour('parsed_timestamp'))
      .withColumn('minute', minute('parsed_timestamp'))
      )
df.show()

+---+-----------+----------+----------+-------------------+---------------+---------------+---------------+-------------------+----+-----+----+----+------+
| id|   date_iso|  date_dmy|  date_mdy|          timestamp|parsed_date_iso|parsed_date_dmy|parsed_date_mdy|   parsed_timestamp|year|month| day|hour|minute|
+---+-----------+----------+----------+-------------------+---------------+---------------+---------------+-------------------+----+-----+----+----+------+
|  1| 2023-01-15|15/01/2023|01/15/2023|2023-01-15 10:30:00|     2023-01-15|     2023-01-15|     2023-01-15|2023-01-15 10:30:00|2023|    1|  15|  10|    30|
|  2| 2023-05-20|20/05/2023|05/20/2023|2023-05-20 15:45:00|     2023-05-20|     2023-05-20|     2023-05-20|2023-05-20 15:45:00|2023|    5|  20|  15|    45|
|  3|InvalidDate|31/02/2023|02/31/2023|   InvalidTimestamp|           NULL|           NULL|           NULL|               NULL|NULL| NULL|NULL|NULL|  NULL|
|  4|       NULL|      NULL|      NULL|               NULL|     

In [29]:
df = df.withColumn('days_difference', datediff(current_date(), 'parsed_date_iso'))
df.select('parsed_date_mdy', 'parsed_date_iso', 'days_difference').show()

+---------------+---------------+---------------+
|parsed_date_mdy|parsed_date_iso|days_difference|
+---------------+---------------+---------------+
|     2023-01-15|     2023-01-15|            803|
|     2023-05-20|     2023-05-20|            678|
|           NULL|           NULL|           NULL|
|           NULL|           NULL|           NULL|
+---------------+---------------+---------------+

